In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
from torch.autograd import Variable

In [9]:
sequence_length = 1
input_size = 22
hidden_size = 200
num_layers = 6
num_classes = 3
batch_size = 100
num_epochs = 20
learning_rate = 0.001

dataset = np.loadtxt("alpine-1.csv", delimiter=',', skiprows=1)
sensors=dataset[:,3:]
actions=dataset[:,:3]
sensors_train=torch.from_numpy(sensors).float()
actions_train=torch.from_numpy(actions).float()
ds = torch.utils.data.TensorDataset(sensors_train, actions_train)
train_loader = torch.utils.data.DataLoader(dataset=ds,batch_size=batch_size,shuffle=True)

In [10]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

rnn = RNN()
rnn.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [12]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size)).cuda()  
        labels = Variable(labels).cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, len(sensors), loss.data[0]))


Epoch [1/20], Step [100/17046], Loss: 0.0118
Epoch [2/20], Step [100/17046], Loss: 0.0360
Epoch [3/20], Step [100/17046], Loss: 0.0260
Epoch [4/20], Step [100/17046], Loss: 0.0253
Epoch [5/20], Step [100/17046], Loss: 0.0144
Epoch [6/20], Step [100/17046], Loss: 0.0181
Epoch [7/20], Step [100/17046], Loss: 0.0322
Epoch [8/20], Step [100/17046], Loss: 0.0059
Epoch [9/20], Step [100/17046], Loss: 0.0483
Epoch [10/20], Step [100/17046], Loss: 0.0181
Epoch [11/20], Step [100/17046], Loss: 0.0291
Epoch [12/20], Step [100/17046], Loss: 0.0258
Epoch [13/20], Step [100/17046], Loss: 0.0251
Epoch [14/20], Step [100/17046], Loss: 0.0238
Epoch [15/20], Step [100/17046], Loss: 0.0103
Epoch [16/20], Step [100/17046], Loss: 0.0278
Epoch [17/20], Step [100/17046], Loss: 0.0148
Epoch [18/20], Step [100/17046], Loss: 0.0244
Epoch [19/20], Step [100/17046], Loss: 0.0118
Epoch [20/20], Step [100/17046], Loss: 0.0202


In [15]:
# Save the Model
torch.save(rnn.state_dict(), 'rnn.pkl')